<a href="https://colab.research.google.com/github/Alonge9500/LLMZoomcamp/blob/main/homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install torch torchvision transformers


In [17]:
import pandas as pd
import numpy as np

In [3]:
url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{url}?raw=1'
df = pd.read_csv(url)

df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [4]:
df = df.iloc[:300]

In [11]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
# Question 1
answer_llm = df.iloc[0].answer_llm
embedding_model.encode(answer_llm)

-0.42244658

In [19]:
#Question 2
evaluations = []
llm_answer = df['answer_llm'].tolist()
orig_answer = df['answer_orig'].tolist()

for i in range(len(llm_answer)):
  llm_answerr = embedding_model.encode(llm_answer[i])
  orig_answerr = embedding_model.encode(orig_answer[i])

  evaluations.append(np.dot(llm_answerr, orig_answerr))

df['evaluation'] = evaluations



30.878217887878417

In [20]:
np.percentile(df['evaluation'], 75)

31.674307823181152

In [27]:
#Question 3
def cosine_eval(v):
  norm = np.sqrt((v * v).sum())
  v_norm = v / norm

  return v_norm

In [28]:
cosine_evaluations = []

for i in range(len(llm_answer)):
  llm_answerr = cosine_eval(embedding_model.encode(llm_answer[i]))
  orig_answerr = cosine_eval(embedding_model.encode(orig_answer[i]))

  cosine_evaluations.append(np.dot(llm_answerr, orig_answerr))

df['cosine_evaluations'] = cosine_evaluations

In [29]:
np.percentile(df['cosine_evaluations'], 75)

0.8362347930669785

In [30]:
!pip install rouge


In [33]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10]
#Question 4
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [44]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [39]:
# Question 5
(scores['rouge-1']['f'] + scores['rouge-2']['f'] +scores['rouge-l']['f']) / 3

0.35490034990035496

In [40]:
df['rouge-1'] = df.apply(lambda x: (rouge_scorer.get_scores(x['answer_llm'], x['answer_orig'])[0]['rouge-1']['f']), axis=1)
df['rouge-2'] = df.apply(lambda x: (rouge_scorer.get_scores(x['answer_llm'], x['answer_orig'])[0]['rouge-2']['f']), axis=1)
df['rouge-l'] = df.apply(lambda x: (rouge_scorer.get_scores(x['answer_llm'], x['answer_orig'])[0]['rouge-l']['f']), axis=1)

In [43]:
#Question 6
df['rouge-2'].mean()

0.20696501983423318